In [38]:
from tictacchess import *


scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+scores['Empate']*empates
    return tabelaScore

def jogaNpares(jogo,n,jog1,jog2):
    name_jog1=jog1.__name__
    name_jog2=jog2.__name__
    tabelaPrim={name_jog1:0, name_jog2:0, 'Empate':0}
    tabelaSeg={name_jog1:0, name_jog2:0, 'Empate':0}
    tabela={}
    for i in range(n):
        #_,_,vencedor=jogo.jogar(jog1,jog2,verbose=False)
        vencedor=jogo.jogar(jog1,jog2,verbose=False)

        if vencedor>0:
            vencedor=name_jog1
        elif vencedor<0:
            print(str(i) + ":PRETAS")
            vencedor=name_jog2
        else:
            vencedor='Empate'
        tabelaPrim[vencedor]+=1
        vencedor=jogo.jogar(jog2,jog1,verbose=False)
        if vencedor>0:
            print(str(i) + ":BRANCAS")
            vencedor=name_jog2
        elif vencedor<0:
            vencedor=name_jog1
        else:
            vencedor='Empate'
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela)

def func_pecas(estado,jogador) :
    clone=copy.deepcopy(estado)
    n_pecas_jogador = len(clone.player_used_pieces(clone.to_move))
    n_pecas_adversario = len(clone.player_used_pieces(clone.other()))
    return n_pecas_jogador - n_pecas_adversario

def func_tactic(estado,jogador) :
    clone=copy.deepcopy(estado)
    winner = clone.have_winner()
    if winner != None:
        return infinity if winner==jogador else -infinity
    # se não reconhece o final do jogo, verifica quem tem três em linha:
    almost_winner = clone.n_in_row(3)
    if almost_winner == None or almost_winner == 'BOTH':
        return 0
    return 1 if almost_winner==jogador else -1


def func_tactic_e_pecas(estado,jogador):
    return func_tactic(estado,jogador) + func_pecas(estado,jogador)

def jogador_tactic_e_pecas_3(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=func_tactic_e_pecas)

def jogador_tactic_3(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=func_tactic)

def jogador_tactic_247(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=func_247)


def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]

def torneio(n,jogadores):
    jogo=TicTacChess()
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            print(jog1.__name__,'vs',jog2.__name__)
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    #return tabela
    print(dict(sorted(tabela.items(), key=lambda x: x[1],reverse=True)))


In [ ]:
def func_247(state, player):
    """A player that chooses a legal move at random."""
    clone=copy.deepcopy(state)
    return func_winner(clone, player) + func_lines(clone, player) #+ func_pieces_value(state,player)

def func_lines(state, player):
    pieces_cells = state.player_used_cells(player)[0]
    pieces = state.player_used_pieces(player)
    other_pieces = state.player_used_cells(state.other())[0]
    num_pieces = len(pieces_cells)
    center = [(1,1),(1,2),(2,1),(2,2)]

    score = 0
    # Check if the piece is in the center
    ##if num_pieces == 2:
    ##    score += 3 if pieces_cells[0] in center and pieces[0] in ['c','C'] else -1

    # Check if two pieces are in a row and not blocked by opponent
    if num_pieces == 3:
        if in_row(pieces_cells) and all(not in_row([pieces_cells[0], p]) for p in other_pieces):
            score += 10
        else:
            score -= 5

    # Additional heuristic: prefer having more pieces on the board
    score += num_pieces

    return score

def func_pieces_value(state,player):
    pieces = state.player_used_pieces(player)
    other_pieces = state.player_used_cells(state.other())[0]
    pieces_cells = state.player_used_cells(player)[0]
    num_pieces = len(pieces_cells)

    center = [(1,1),(1,2),(2,1),(2,2)]
    walls = [(0,1),(0,2),(1,0),(1,3),(2,0),(2,3),(3,1),(3,2)]
    corners = [(0,0),(0,3),(3,0),(3,3)]
    score = 0
    for piece in pieces:
        for cell in pieces_cells:
            if cell in center:
                if piece in ['p', 'P']:
                    score += 1
                elif piece in ['c', 'C']:
                    score += 3
                elif piece in ['b', 'B']:
                    score += 3
                elif piece in ['t', 'T']:
                    score += 5
            elif cell in walls:
                if piece in ['p', 'P']:
                    score += 1
                elif piece in ['c', 'C']:
                    score += 2
                elif piece in ['b', 'B']:
                    score += 3
                elif piece in ['t', 'T']:
                    score += 5
            elif cell in corners:
                if piece in ['c', 'C']:
                    score += 1

    return score

#def func_black_pieces(state,player):

def func_winner(state, player):
    winner = state.have_winner()
    if winner != None:
        return infinity if winner == player else -infinity
    # se não reconhece o final do jogo, verifica quem tem três em linha:
    almost_winner = state.n_in_row(3)
    if almost_winner == player:
        return 100
    elif almost_winner == state.other():
        # Check if current state is blocking the row of the other player
        if row_blocked(state, 3,player, state.other()):
            print("Bloqueou")
            return 70
        return -100
    elif almost_winner == "BOTH":
        used_pieces = state.player_used_pieces(player)
        # Check if current state is blocking the row of the other player
        if row_blocked(state, 3,player, state.other()):
            print("Bloqueou")
            return 70
        if len(used_pieces) == 3:
            available_pieces = [p for p in state.player_pieces(player) if p not in used_pieces]
            for piece in available_pieces:
                possible_moves = state.empty_cells()
                for move in possible_moves:
                    # Cria uma cópia do estado e aplica o movimento
                    clone = state.next_state(move)
                    # Verifica se o movimento resulta em vitória para o jogador
                    if clone.have_winner() == player:
                        return 100
        other_no_win = True
        for piece in used_pieces:  # Todas as peças do jogador
            possible_moves = state.possible_moves(piece)
            for move in possible_moves:
                # Cria uma cópia do estado e aplica o movimento
                clone = state.next_state(move)
                # Verifica se o movimento resulta em vitória para o jogador
                if clone.have_winner() == player:
                    return 100

                # Verifica se o movimento bloqueia uma vitória do oponente
                if clone.have_winner() == state.other():
                    other_no_win = False

        if not other_no_win:
            return -70
    return 0

def row_blocked(state, n, player, other):
    "Return the player or players who have n-in-row (maybe Both), or None."
    #(play,board,jogadas,capturas,pawn_direction,piece)=self
    cells1,_ = state.player_used_cells(player)
    cells2,_ = state.player_used_cells(other)
    print(cells1)
    print(cells2)
    is_blocked = False
    print("")
    if len(cells2) < n:
        return False
    if len(cells2) == n:
        cells2_edit = sorted(cells2)
        for cell1 in cells1:
            cells2_edit.append(cell1)
            if in_row(cells2_edit):
                is_blocked = True
                break
            cells2_edit.remove(cell1)
    else:
        if len(cells2) > n: # we check if a group of them is in_row, assume
            for group_cells2 in combinations(cells2, n):
                group_cells2 = sorted(group_cells2)
                for cell1 in cells1:
                    group_cells2.append(cell1)
                    print(group_cells2)
                    if in_row(group_cells2):
                        is_blocked = True
                        break
                    group_cells2.remove(cell1)
    return is_blocked


jogo=TicTacChess()
torneio(2,[jogador_tactic_3, jogador_tactic_247])

jogador_tactic_3 vs jogador_tactic_247
0:PRETAS
1:BRANCAS
{'jogador_tactic_3': 6, 'jogador_tactic_247': 6}


In [ ]:
def func_winner(state, player):
    winner = state.have_winner()
    if winner != None:
        return infinity if winner == player else -infinity
    # se não reconhece o final do jogo, verifica quem tem três em linha:
    almost_winner = state.n_in_row(3)
    if almost_winner == None:
        return 0
    elif almost_winner == player:
        return 20
    elif almost_winner == state.other():
        #validate if the player is blocking the other player



        return -20

    # Check if the player can win in the next move
    if almost_winner == "BOTH":
        used_pieces = state.player_used_pieces(player)
        if len(used_pieces) == 3:
            available_pieces = [p for p in state.player_pieces(player) if p not in used_pieces]
            for piece in available_pieces:
                possible_moves = state.empty_cells("")
                for move in possible_moves:
                    # Cria uma cópia do estado e aplica o movimento
                    clone = state.next_state(move)
                    # Verifica se o movimento resulta em vitória para o jogador
                    if clone.have_winner() == player:
                        return 20
        other_no_win = True
        for piece in used_pieces:  # Todas as peças do jogador
            possible_moves = state.possible_moves(piece)
            for move in possible_moves:
                # Cria uma cópia do estado e aplica o movimento
                clone = state.next_state(move)
                # Verifica se o movimento resulta em vitória para o jogador
                if clone.have_winner() == player:
                    return 15

                # Verifica se o movimento bloqueia uma vitória do oponente
                if clone.have_winner() == state.other():
                    other_no_win = False

        if not other_no_win:
            return -15
    return 0




print(row_blocked(EstadoTicTacChess(to_move="test",board={'B': (0, 2), 'c': (3, 2), 'P': (1, 2), 't': (3, 3), 'T': (2, 2), 'p': (0, 3), 'b': (2, 0), 'C': (1,3)},n_jogadas=10,n_capturas=0,pawn_direction="1",last_piece="1")
, 3, "test"))

[(3, 2), (3, 3), (0, 3), (2, 0)]
[(0, 2), (1, 2), (2, 2), (1, 3)]

[(0, 2), (1, 2), (2, 2), (3, 2)]
[(0, 2), (1, 2), (1, 3), (3, 2)]
[(0, 2), (1, 2), (1, 3), (3, 3)]
[(0, 2), (1, 2), (1, 3), (0, 3)]
[(0, 2), (1, 2), (1, 3), (2, 0)]
[(0, 2), (1, 3), (2, 2), (3, 2)]
[(0, 2), (1, 3), (2, 2), (3, 3)]
[(0, 2), (1, 3), (2, 2), (0, 3)]
[(0, 2), (1, 3), (2, 2), (2, 0)]
[(1, 2), (1, 3), (2, 2), (3, 2)]
[(1, 2), (1, 3), (2, 2), (3, 3)]
[(1, 2), (1, 3), (2, 2), (0, 3)]
[(1, 2), (1, 3), (2, 2), (2, 0)]
True
